# Calculation Gradient
   - Tracking the history of the computation

In [1]:
"""import modules"""
import torch

In [2]:
"""create random tensor"""
x = torch.randn(3, requires_grad = True) # requires_grad = True is essential to create gradient function
print("x is:\n",x)

x is:
 tensor([-1.2643,  1.0159,  0.2505], requires_grad=True)


In [3]:
"""example of gradient functions"""
y = x + 2
print("y is:\n",y) # return AddBackward, because the operator is addition

y is:
 tensor([0.7357, 3.0159, 2.2505], grad_fn=<AddBackward0>)


**Note:** this gradient function can be used to find the gradient value of y with respect to x: backpropagation 

In [4]:
z = y*y*2
print("z is:\n",z) # return MulBackward, because the operator is multiplication

z is:
 tensor([ 1.0825, 18.1908, 10.1292], grad_fn=<MulBackward0>)


In [5]:
z = z.mean() # return an scalar output 
print("z is:\n",z) # return MeanBackward, because the operator is MeanBackward

z is:
 tensor(9.8008, grad_fn=<MeanBackward0>)


In [6]:
"""gradient of recent z with respect to x"""
z.backward() # dz/dx, no parameter value is necessary
print("gradient value: ",x.grad)

tensor([0.9809, 4.0211, 3.0006])


**Note:** grad can be produced only for scalar output, otherwise it shows an error. Such as,

In [7]:
"""grad or backward is not possible for non scalar outputs"""
z = y*y*2 # don't return a scalar output
print("z is:\n",z)

z.backward() # dz/dx
print(x.grad)

z is:
 tensor([ 1.0825, 18.1908, 10.1292], grad_fn=<MulBackward0>)


RuntimeError: grad can be implicitly created only for scalar outputs

**Note:** in order to solve the problem, we need to use a vector with same size as output, in the function backward as a parameter. Such as,

In [10]:
"""grad or backward for non scalar outputs"""
z = y*y*2 # don't return a scalar output
print("z is:\n",z)

v = torch.tensor([0.1, 1.0, 0.001], dtype = torch.float32) # vector with same size as output

z.backward(v) # dz/dx, use the vector as parameter value
print("\ngradient value: ",x.grad)

z is:
 tensor([ 1.0825, 18.1908, 10.1292], grad_fn=<MulBackward0>)

gradient value:  tensor([ 1.8638, 40.2115,  3.0276])


**Prevent Tesnor from Calculating the Gradient, by using three methods:**
   - assume we have a tensor x, then:
       - x.requires_grad_(False)
       - x.detach()
       - with torch.no_grad():

In [11]:
"""create random tensor"""
x = torch.randn(3, requires_grad = True) # requires_grad = True is essential to create gradient function
print("x is:\n",x)

x is:
 tensor([-0.8556,  0.9844,  2.5797], requires_grad=True)


In [12]:
"""using inplace operator"""
x.requires_grad_(False)
print("x is:\n",x)

x is:
 tensor([-0.8556,  0.9844,  2.5797])


In [13]:
"""by making a vector"""
y = x.detach()
print("y is:\n",y)

y is:
 tensor([-0.8556,  0.9844,  2.5797])


In [14]:
"""by torch statement"""
with torch.no_grad():
    y = x+2
print("y is:\n",y)

y is:
 tensor([1.1444, 2.9844, 4.5797])


**Note:** we must empty our gradient in the training loop (for each steps), otherwise we will get wrong outcomes.

In [16]:
"""wrong approach"""
weights = torch.ones(4, requires_grad = True)

for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print("gradient value in the epoch {} is: {}".format(epoch, weights.grad))

gradient value in the epoch 0 is: tensor([3., 3., 3., 3.])
gradient value in the epoch 1 is: tensor([6., 6., 6., 6.])
gradient value in the epoch 2 is: tensor([9., 9., 9., 9.])


In [18]:
"""right approach"""
weights = torch.ones(4, requires_grad = True)

for epoch in range(3):
    model_output = (weights * 3).sum()
    model_output.backward()
    print("gradient value in the epoch {} is: {}".format(epoch, weights.grad))
    weights.grad.zero_() # set the gradient value to zero after each loop

gradient value in the epoch 0 is: tensor([3., 3., 3., 3.])
gradient value in the epoch 1 is: tensor([3., 3., 3., 3.])
gradient value in the epoch 2 is: tensor([3., 3., 3., 3.])


**Note:** we can use optimizer to calculate gradient value, but for each cases, we must set the gradient to zero after each iteration.